## Import relevant libraries and functions

In [7]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator, 
    perform_experiments,
    Samplers, Constant,
    RealParameter, Scenario
)
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.analysis import (feature_scoring, pairs_plotting)

from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

## Prepare model and parameters

In [8]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2)

# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [9]:
# defining specific policies

def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

zeropolicy = Policy("zero policy",**dict(get_do_nothing_dict()))

## Open experiments

In [10]:
with open('experiments_SOBOL_1000x30', 'rb') as file:
    experiments_SOBOL_OE_SA = pickle.load(file)
    
with open('outcomes_SOBOL_1000x30', 'rb') as file:
    outcomes_SOBOL_OE_SA = pickle.load(file)

## Scenario Discovery

In [11]:
outcomes_SOBOL_OE_SA

{'Expected Annual Damage': array([0., 0., 0., ..., 0., 0., 0.]),
 'Dike Investment Costs': array([6.72353782e+08, 6.72353782e+08, 6.72353782e+08, ...,
        6.50368937e+08, 6.50368937e+08, 6.50368937e+08]),
 'RfR Investment Costs': array([1.3379e+09, 1.3379e+09, 1.3379e+09, ..., 1.2245e+09, 1.2245e+09,
        1.2245e+09]),
 'Evacuation Costs': array([0., 0., 0., ..., 0., 0., 0.]),
 'Expected Number of Deaths': array([0., 0., 0., ..., 0., 0., 0.])}

In [12]:
problem = get_SALib_problem(uncertainties)
results = {}
for i in outcomes_SOBOL_OE_SA:
    print(i)
    Si = sobol.analyze(problem, outcomes_SOBOL_OE_SA[i], calc_second_order=True, print_to_console=True)
    results[i] = Si


Expected Annual Damage


C:\Users\Asus\Documents\Python venvs\MBDMv2\Lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                                   ST       ST_conf
A.0_ID flood wave shape  6.344984e-03  7.191533e-04
A.1_Bmax                 2.727820e-04  2.311709e-04
A.1_Brate                3.558831e-06  6.148996e-06
A.1_pfail                3.336354e-01  2.481481e-02
A.2_Bmax                 5.896626e-05  4.944574e-05
A.2_Brate                1.639663e-06  1.048402e-06
A.2_pfail                1.761456e-01  1.343678e-02
A.3_Bmax                 7.950504e-06  5.665754e-06
A.3_Brate                7.744179e-07  5.580692e-07
A.3_pfail                3.139702e-01  2.125417e-02
A.4_Bmax                 2.561704e-05  1.641731e-05
A.4_Brate                1.335162e-06  6.856500e-07
A.4_pfail                2.472599e-03  2.947073e-04
A.5_Bmax                 0.000000e+00  0.000000e+00
A.5_Brate                0.000000e+00  0.000000e+00
A.5_pfail                5.926904e-02  6.492389e-03
discount rate 0          3.753215e-02  3.273655e-03
discount rate 1          8.521419e-03  1.301815e-03
discount rat

C:\Users\Asus\Documents\Python venvs\MBDMv2\Lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                          ST  ST_conf
A.0_ID flood wave shape  0.0      0.0
A.1_Bmax                 0.0      0.0
A.1_Brate                0.0      0.0
A.1_pfail                0.0      0.0
A.2_Bmax                 0.0      0.0
A.2_Brate                0.0      0.0
A.2_pfail                0.0      0.0
A.3_Bmax                 0.0      0.0
A.3_Brate                0.0      0.0
A.3_pfail                0.0      0.0
A.4_Bmax                 0.0      0.0
A.4_Brate                0.0      0.0
A.4_pfail                0.0      0.0
A.5_Bmax                 0.0      0.0
A.5_Brate                0.0      0.0
A.5_pfail                0.0      0.0
discount rate 0          0.0      0.0
discount rate 1          0.0      0.0
discount rate 2          0.0      0.0
                          S1  S1_conf
A.0_ID flood wave shape  0.0      0.0
A.1_Bmax                 0.0      0.0
A.1_Brate                0.0      0.0
A.1_pfail                0.0      0.0
A.2_Bmax                 0.0      0.0
A.2_Brate   

...

                          ST  ST_conf
A.0_ID flood wave shape  0.0      0.0
A.1_Bmax                 0.0      0.0
A.1_Brate                0.0      0.0
A.1_pfail                0.0      0.0
A.2_Bmax                 0.0      0.0
A.2_Brate                0.0      0.0
A.2_pfail                0.0      0.0
A.3_Bmax                 0.0      0.0
A.3_Brate                0.0      0.0
A.3_pfail                0.0      0.0
A.4_Bmax                 0.0      0.0
A.4_Brate                0.0      0.0
A.4_pfail                0.0      0.0
A.5_Bmax                 0.0      0.0
A.5_Brate                0.0      0.0
A.5_pfail                0.0      0.0
discount rate 0          0.0      0.0
discount rate 1          0.0      0.0
discount rate 2          0.0      0.0
                          S1  S1_conf
A.0_ID flood wave shape  0.0      0.0
A.1_Bmax                 0.0      0.0
A.1_Brate                0.0      0.0
A.1_pfail                0.0      0.0
A.2_Bmax                 0.0      0.0
A.2_Brate   

C:\Users\Asus\Documents\Python venvs\MBDMv2\Lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                                   ST       ST_conf
A.0_ID flood wave shape  8.111273e-03  9.292061e-04
A.1_Bmax                 2.276803e-04  1.845005e-04
A.1_Brate                2.326341e-06  4.047926e-06
A.1_pfail                1.539614e-02  2.051177e-03
A.2_Bmax                 1.133031e-04  1.436345e-04
A.2_Brate                5.245625e-06  6.397475e-06
A.2_pfail                3.331488e-01  2.658112e-02
A.3_Bmax                 5.624227e-06  5.056483e-06
A.3_Brate                8.966741e-07  9.671377e-07
A.3_pfail                3.503851e-01  3.108224e-02
A.4_Bmax                 7.657303e-05  5.407652e-05
A.4_Brate                4.310360e-06  3.189606e-06
A.4_pfail                6.548505e-03  7.707551e-04
A.5_Bmax                 0.000000e+00  0.000000e+00
A.5_Brate                0.000000e+00  0.000000e+00
A.5_pfail                1.013967e-01  8.916537e-03
discount rate 0          0.000000e+00  0.000000e+00
discount rate 1          0.000000e+00  0.000000e+00
discount rat

C:\Users\Asus\Documents\Python venvs\MBDMv2\Lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                                   ST       ST_conf
A.0_ID flood wave shape  7.801914e-03  1.225264e-03
A.1_Bmax                 4.152601e-05  4.354896e-05
A.1_Brate                4.048232e-07  6.100387e-07
A.1_pfail                2.113789e-02  3.916564e-03
A.2_Bmax                 3.510848e-05  2.194921e-05
A.2_Brate                1.057768e-06  6.278699e-07
A.2_pfail                6.879420e-02  9.199043e-03
A.3_Bmax                 1.662762e-05  1.341212e-05
A.3_Brate                5.813518e-07  3.975396e-07
A.3_pfail                6.579953e-01  3.012792e-02
A.4_Bmax                 4.476841e-06  3.160415e-06
A.4_Brate                2.939072e-07  3.390277e-07
A.4_pfail                5.823701e-04  2.346859e-04
A.5_Bmax                 0.000000e+00  0.000000e+00
A.5_Brate                0.000000e+00  0.000000e+00
A.5_pfail                5.138403e-02  7.063705e-03
discount rate 0          0.000000e+00  0.000000e+00
discount rate 1          0.000000e+00  0.000000e+00
discount rat

In [16]:
def sum_three_arrays_in_dict(dictionary, key1, key2, key3, result_key):
    if key1 in dictionary and key2 in dictionary and key3 in dictionary:
        array1 = dictionary[key1]
        array2 = dictionary[key2]
        array3 = dictionary[key3]
        
        if isinstance(array1, np.ndarray) and isinstance(array2, np.ndarray) and isinstance(array3, np.ndarray):
            if array1.shape == array2.shape == array3.shape:
                dictionary[result_key] = array1 + array2 + array3
            else:
                raise ValueError(f"Arrays under keys '{key1}', '{key2}', and '{key3}' are not of the same shape")
        else:
            raise TypeError(f"Values under keys '{key1}', '{key2}', and '{key3}' are not NumPy arrays")
    else:
        raise KeyError(f"Keys '{key1}', '{key2}', and/or '{key3}' not found in the dictionary")

    return dictionary

## Dit aanpassen naar top 10% ofzo

In [26]:
top_10 = int(0.1*len(outcomes_SOBOL_OE_SA['Expected Annual Damage']))

120000


In [28]:
y_damages = outcomes_SOBOL_OE_SA['Expected Annual Damage'].sort_values().iloc[top_10]
y_deaths = outcomes_SOBOL_OE_SA['Expected Number of Deaths'].sort_values().iloc[top_10]
new_cost_dict = sum_three_arrays_in_dict(outcomes_SOBOL_OE_SA, 'Dike Investment Costs', 'RfR Investment Costs', 'Evacuation Costs', 'Total Costs')
y_costs = new_cost_dict['Total Costs'].sort_values().iloc[top_10]

AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'

In [19]:
x = experiments_SOBOL_OE_SA

## PRIM

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)
prim_alg = prim.Prim(x=x, y=y_damages, threshold= 0.8, peel_alpha= 0.1)

box_damages = prim_alg.find_box()

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)
prim_alg = prim.Prim(x=x, y=y_deaths, threshold= 0.8, peel_alpha= 0.1)

box_deaths = prim_alg.find_box()

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)
prim_alg = prim.Prim(x=x, y=y_costs, threshold= 0.8, peel_alpha= 0.1)

box_costs = prim_alg.find_box()